In [1]:
import pandas as pd
import numpy as np

In [2]:
client = "cooptotal"
df = pd.read_excel(f"./{client}_results.xlsx")
df

,hostname,Impressions,Inviews,Fixations,Fixation/Inview Ratio,Average Fixation Duration,Total Fixation Duration,Client Total Fixation Duration,Sample Size
0,aftonbladet.se,1387320,1053056,572270,54.34,741.92,73450.0,4.245786e+08,high
1,blocket.se,4058,3342,1691,50.60,0.00,0.0,0.000000e+00,low
2,bloggar.aftonbladet.se,155594,110212,78661,71.37,3396.35,472093.0,2.671603e+08,medium
3,klart.se,7023,4510,1882,41.73,1982.78,729662.0,3.731592e+06,medium
4,malservice.aftonbladet.se,11,6,2,33.33,643.23,7075.5,1.286460e+03,low
5,skonhetsredaktorerna.se,86770,66274,86770,130.93,0.00,0.0,0.000000e+00,low
6,svd.se,98759,79722,42847,53.75,8411.37,42519482.0,3.604020e+08,medium
7,tramsfrans.aftonbladet.se,86770,66274,0,0.00,NaN,0.0,NaN,low
8,tv.nu,17359,12685,4676,36.86,2310.06,505903.0,1.080184e+07,low


In [5]:
df = df[df["Sample Size"].isin(["high", "medium"])]
df["group"] = 1

/tmp/ipykernel_8927/2862922107.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["group"] = 1


In [6]:
final_df = df.groupby(["group"]).apply(
    lambda x: pd.Series(
        {
            "Impressions": np.floor(np.average(x["Impressions"], weights=x["Impressions"])),
            "Inviews": np.floor(np.average(x["Inviews"], weights=x["Inviews"])),
            "Fixations": np.floor(np.average(x["Fixations"], weights=x["Fixations"])),
            "Fixation/Inview Ratio": round(np.average(x["Fixation/Inview Ratio"], weights=x["Impressions"]), 2),
            "Average Fixation Duration": np.floor(np.average(x["Average Fixation Duration"], weights=x["Fixations"])),
            "Total Fixation Duration": np.floor(np.average(x["Average Fixation Duration"], weights=x["Fixations"]) *  np.floor(np.average(x["Fixations"], weights=x["Fixations"])))
        }
    )
)
final_df = final_df.reset_index()
final_df

,group,Impressions,Inviews,Fixations,Fixation/Inview Ratio,Average Fixation Duration,Total Fixation Duration
0,1,1387320.0,1053056.0,572270.0,54.34,741.0,424578558.0


In [7]:
final_df = final_df[["Impressions", "Inviews", "Fixations", "Fixation/Inview Ratio", "Average Fixation Duration", "Total Fixation Duration"]]

final_df.to_excel(f"./{client}_results_consolidated.xlsx", index=False)